# Jarvislabs

> support for Jarvislabs

In [2]:
#| default_exp jarvislabs

In [3]:
#| hide
from nbdev.showdoc import *

In [106]:
#| hide
import re
def print_safe(s, _pat=re.compile('jarvislabs.ai/[-_/a-zA-Z0-9?.=]+')):
    print(_pat.sub('jarvislabs.ai/...', str(s)))
    
print_safe('''0, sd:  A5000 (1) - Paused
	# https://notebooksg.jarvislabs.ai/062/lab?token=super-secret-token_123/
	# ('sshg.jarvislabs.ai', 8962''')

0, sd:  A5000 (1) - Paused
	# https://notebooksg.jarvislabs.ai/...
	# ('sshg.jarvislabs.ai', 8962


In [4]:
#| export
import re
import os
from pathlib import Path

from fastcore.script import *
from fastcore.basics import *
from fastcore.imports import *

In [5]:
#| export
def _jl_parse_ssh_str(ssh_str, pat=re.compile(r"ssh -p *(\d+) root@(.*)")):
    m = pat.search(ssh_str)
    if not m:
        raise ValueError(f"Unrecognised jarvis ssh string: {ssh_str}")
    return m.group(2), int(m.group(1))

In [6]:
_jl_parse_ssh_str('ssh -p 3323 root@test.jarvislabs.com')

('test.jarvislabs.com', 3323)

In [119]:
#| export
def _jl_print_inst(inst, idx="", show_conn_info=True):
    addr = ', '.join(filter(lambda x: x, [f'{idx}', inst.name]))
    print(f"{addr}:  {inst.gpu_type} ({inst.num_gpus}) - {inst.status}")
    if show_conn_info:
        print(f"\t# {inst.url}")
        print(f"\t# {_jl_parse_ssh_str(inst.ssh_str) if inst.ssh_str else 'No SSH info, did you provide ssh key?'}")

In [62]:
from jlclient.jarvisclient import Instance

inst = Instance(gpu_type='v100', name="my-test", num_gpus=1, status='paused', url='https://.../?token=...', ssh_str='ssh -p 1234 root@test.jarvislabs.com', hdd=0, framework_id='fastai', machine_id=1, tboard_url='https://...')
_jl_print_inst(inst)

my-test:  v100 (1) - paused
	# https://.../?token=...
	# ('test.jarvislabs.com', 1234)


In [110]:
#| export

def _jl_make_ssh_config(inst, idx):
    name = inst.name or f'jl-{idx}'
    if not inst.ssh_str:
        return """# no ssh info available"""
    host, port = _jl_parse_ssh_str(inst.ssh_str)
    return f"""
    Host {name}
        Port {port}
        HostKeyAlias {name}
        ForwardAgent yes
        Hostname {host}
        User root
    """

In [111]:
print(_jl_make_ssh_config(inst, 0))


    Host my-test
        Port 1234
        HostKeyAlias my-test
        ForwardAgent yes
        Hostname test.jarvislabs.com
        User root
    


In [112]:
#| export
class JarvisClient():
    def __init__(self, ssh_d=None):
        self.ssh_d = ssh_d or Path.home()/'.ssh'
        self.ssh_config = self.ssh_d/'config'

    def _instances(self):
        from jlclient import jarvisclient as api
        api.token = os.environ['JARVIS_TOKEN']
        api.user_id = os.environ['JARVIS_USER_ID']
        try:
            ids = {str(idx): i for idx, i in enumerate(api.User.get_instances())}
        except AttributeError as e:
            if "'str' object has no attribute 'items'" in str(e):
                print('Api is not listing your instances correctly, check your token or contact jarvislabs support')
                return {},{}
        names = {inst.name: inst for inst in ids.values()}
        return ids, names


In [120]:
token=!gopass -o jarvislabs-test/token
user_id=!gopass -o jarvislabs-test/user_id
os.environ['JARVIS_TOKEN'] = token[0]
os.environ['JARVIS_USER_ID'] = user_id[0]

def tst_client(): return JarvisClient(ssh_d=Path('/tmp/tst.ssh/'))

In [121]:
{k:inst.name for k, inst in tst_client()._instances()[0].items()}

{'0': 'test'}

In [122]:
{k:inst.name for k, inst in tst_client()._instances()[1].items()}

{'test': 'test'}

In [123]:
print_safe(tst_client()._instances())

({'0': {'gpu_type': 'CPU', 'num_gpus': 1, 'hdd': '20', 'framework_id': '0', 'url': 'https://notebooksa.jarvislabs.ai/...', 'machine_id': 61541, 'tboard_url': ['https://notebooksa.jarvislabs.ai/...'], 'ssh_str': '', 'status': 'Paused', 'name': 'test', 'arguments': '', 'is_reserved': 1, 'duration': 'hour'}}, {'test': {'gpu_type': 'CPU', 'num_gpus': 1, 'hdd': '20', 'framework_id': '0', 'url': 'https://notebooksa.jarvislabs.ai/...', 'machine_id': 61541, 'tboard_url': ['https://notebooksa.jarvislabs.ai/...'], 'ssh_str': '', 'status': 'Paused', 'name': 'test', 'arguments': '', 'is_reserved': 1, 'duration': 'hour'}})


In [124]:
#| export
@patch
def list(self:JarvisClient):
    ids, _ = self._instances()       
    for key, inst in ids.items():
        _jl_print_inst(inst, key)


In [125]:
tst_client().list()

0, test:  CPU (1) - Paused
	# https://notebooksa.jarvislabs.ai/0962/lab?token=Oq4r9J2lU_-I6RUgXj7h38hh7OJcS5tNT-kHUbwpzqcdKCkk5nBILg-C2MzacXnK
	# No SSH info, did you provide ssh key?


In [126]:
#| export
@patch
def _add_config_d(self: JarvisClient):
    line = 'Include ~/.ssh/config.d/*'
    if not self.ssh_config.exists():
        print(f'Creating {self.ssh_config} for you')
        self.ssh_d.mkdir(exist_ok=True)
        lines = []
    else:
        with self.ssh_config.open('rt') as f:
            lines = f.readlines()
    for l in lines:
        if '~/.ssh/config.d/' in l.strip():
            break
    else:
        if lines:
            print("Adding the following required include to your ~/.ssh/config:")
            print(line)
        with self.ssh_config.open('wt') as f:
            f.writelines([line, '\n', *lines])


In [127]:
jl = tst_client()
! rm -rf /tmp/tst.ssh
jl._add_config_d()
jl._add_config_d()
! echo Content of the .ssh/config
! cat {jl.ssh_config}

Creating /tmp/tst.ssh/config for you
Content of the .ssh/config
Include ~/.ssh/config.d/*


In [128]:
jl = tst_client()
! mkdir -p /tmp/tst.ssh
! rm /tmp/tst.ssh/config
jl._add_config_d()
jl._add_config_d()
! echo Content of the .ssh/config
! cat {jl.ssh_config}

Creating /tmp/tst.ssh/config for you
Content of the .ssh/config
Include ~/.ssh/config.d/*


In [129]:
jl = tst_client()
! mkdir -p /tmp/tst.ssh
! echo '# some lines\n# some more lines' > /tmp/tst.ssh/config
jl._add_config_d()
! echo Content of the .ssh/config
! cat {jl.ssh_config}

Adding the following required include to your ~/.ssh/config:
Include ~/.ssh/config.d/*
Content of the .ssh/config
Include ~/.ssh/config.d/*
# some lines
# some more lines


In [130]:
#| export
@patch
def setup(self: JarvisClient, instances = None):
    p = self.ssh_d / "config.d" 
    p.mkdir(parents=True, exist_ok=True)
    with (p/'jarvis.config').open('w') as f:
        for key, inst in (instances or self._instances()[0]).items():
            f.write(_jl_make_ssh_config(inst, key))


In [131]:
!rm -rf /tmp/tst.ssh/
tst_client().setup()
!ls /tmp/tst.ssh/config.d
!cat /tmp/tst.ssh/config.d/jarvis.config

jarvis.config
# no ssh info available

In [132]:
#| export
@patch
def resume(self: JarvisClient, idx=0):
    ids, names = self._instances()       
    instance = {**ids, **names}[str(idx)]
    instance.resume()
    _jl_print_inst(instance)
    self.setup(ids)


In [133]:
print_safe(tst_client().resume(0))

test:  CPU (1) - Running
	# https://notebooksa.jarvislabs.ai/0962/lab?token=Oq4r9J2lU_-I6RUgXj7h38hh7OJcS5tNT-kHUbwpzqcdKCkk5nBILg-C2MzacXnK
	# No SSH info, did you provide ssh key?
None


In [134]:
#| export
@patch
def pause(self: JarvisClient, idx=0):
    ids, names = self._instances()       
    instance = {**ids, **names}[str(idx)]
    instance.pause()
    _jl_print_inst(instance)
    self.setup(ids)


In [135]:
tst_client().pause()

test:  CPU (1) - Paused
	# https://notebooksa.jarvislabs.ai/0962/lab?token=Oq4r9J2lU_-I6RUgXj7h38hh7OJcS5tNT-kHUbwpzqcdKCkk5nBILg-C2MzacXnK
	# No SSH info, did you provide ssh key?


In [143]:
#| hide
import nbdev; nbdev.nbdev_export()